<a href="https://colab.research.google.com/github/deepika-rana/Song-Generator/blob/master/Song_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Song Generator

A character-level LSTM is contructed with PyTorch to generate love song. The network will train character by character on some text, then generate new text character by character. 

The input used to train this model was from a public dataset on kaggle that has more than 380k lyrics.
**This model will be able to generate new text based on the text from the lyrics!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn).

In [0]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = "/content/drive/My Drive/data/lyrics.csv"
data = pd.read_csv(path)

In [0]:
lyrics_based = data[data['lyrics'].str.contains('love') == True].copy()
title_based = data[data['song'].str.contains('love') == True].copy()
data = pd.concat([lyrics_based, title_based]).drop_duplicates().reset_index(drop=True)
data = title_based.dropna(subset=['lyrics'])
data[:20]

,index,song,year,artist,genre,lyrics
8,8,slow-love,2009,beyonce-knowles,Pop,[Verse 1:]\nI read all of the magazines\nwhile...
9,9,why-don-t-you-love-me,2009,beyonce-knowles,Pop,"N-n-now, honey\nYou better sit down and look a..."
23,23,he-still-loves-me-f-choir-from-fighting-tempta...,2007,beyonce-knowles,Pop,Took me a while but I'm finally here\nSo I jus...
36,36,keep-givin-your-love-to-me,2007,beyonce-knowles,Pop,"Hey yo, B\nThis a good look right here, yeah\n..."
40,40,still-in-love,2007,beyonce-knowles,Pop,I love you\nI love you\nI love you\nBaby I lov...
79,79,drunk-in-love,2013,beyonce-knowles,Pop,"[Intro: BeyoncÃ©]\nI've been drinking, I've be..."
82,82,hair-crazy-in-love,2010,beyonce-knowles,Pop,I love you\nI love you\nI love you\nBaby I lov...
104,104,dangerously-in-love,2006,beyonce-knowles,Pop,"I love you\nI love you, I love you\nBaby, I lo..."
123,123,he-still-loves-me,2006,beyonce-knowles,Pop,Took me a while but I'm finally here\nSo I jus...
124,124,dangerously-in-love-2,2006,beyonce-knowles,Pop,"I love you\nI love you, I love you\nBaby, I lo..."


In [0]:
lyrics = ''.join(data.lyrics)
lyrics = lyrics.lower()
lyrics = ''.join([c for c in lyrics if c not in "#$%&()*+,-./:;<=>@[\\]^_`{|}~"])
lyrics[:200]

"verse 1\ni read all of the magazines\nwhile waiting around\nyou said you couldn't wait to see me\nbut you got stuck in town\nso turn off the phone\nnow we're finally alone\nand some moments at the end of the"

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [0]:
# Creating two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(lyrics))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encoding the text
encoded = np.array([char2int[ch] for ch in lyrics])

And those same characters from above, are encoded as integers.

In [0]:
encoded[:100]

array([ 86,  79,  36,  97,  79, 108,  45, 106,  76, 108,  36,  79,  42,
        62, 108,  42, 125, 125, 108, 103,  11, 108,  44, 122,  79, 108,
        52,  42,  66,  42, 111,  76,   9,  79,  97, 106,  63, 122,  76,
       125,  79, 108,  63,  42,  76,  44,  76,   9,  66, 108,  42,  36,
       103,  21,   9,  62, 106, 127, 103,  21, 108,  97,  42,  76,  62,
       108, 127, 103,  21, 108,   0, 103,  21, 125,  62,   9,  75,  44,
       108,  63,  42,  76,  44, 108,  44, 103, 108,  97,  79,  79, 108,
        52,  79, 106,  20,  21,  44, 108, 127, 103])

## Pre-processing the data

The LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. One-hot encoding the data using a function.


In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [0]:
# Check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, I created mini-batches for training. 

In this example, taking the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of the sequences will be `seq_length` long.

### Creating Batches

**1. The first thing is to discard some of the text so to obtain completely full mini-batches.**

**2. After that, split `arr` into $N$ batches.** 

**3. Now that we have this array, we can iterate through it to get our mini-batches.**


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''A generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Testing the Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [0]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 86  79  36  97  79 108  45 106  76 108]
 [ 86 103 117  79 106 125  76 117  79 108]
 [ 79 106  44  21  36   9 108 127 103  21]
 [ 89 125  79  42  97  79 108  52  79 106]
 [ 44 122  79 106  89 122 103   9  79 108]
 [ 44 108 127 103  21 108 103  21  36 108]
 [106  79  86  79  36 127  20 103  62 127]
 [ 76  36  44 122 106  76  44  97 108 127]]

y
 [[ 79  36  97  79 108  45 106  76 108  36]
 [103 117  79 106 125  76 117  79 108  42]
 [106  44  21  36   9 108 127 103  21  36]
 [125  79  42  97  79 108  52  79 106 122]
 [122  79 106  89 122 103   9  79 108  97]
 [108 127 103  21 108 103  21  36 108 125]
 [ 79  86  79  36 127  20 103  62 127 108]
 [ 36  44 122 106  76  44  97 108 127 103]]


---
## Defining the network with PyTorch

Next, I'll use PyTorch to define the architecture of the network. Started by defining the layers and operations. Then, define a method for the forward pass. 

In [0]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,  
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        # pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below using an Adam optimizer and cross entropy loss since as looking at character class scores as output. Calculate the loss and perform backpropagation.

A couple of details about training: 
>* Within the batch loop, detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Actually training the network. First creating the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [0]:
#Set model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(131, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=131, bias=True)
)


### Set training hyperparameters!

In [19]:
batch_size = 128
seq_length = 100
n_epochs = 20 

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.1058... Val Loss: 3.0621
Epoch: 1/20... Step: 20... Loss: 3.0564... Val Loss: 2.9787
Epoch: 1/20... Step: 30... Loss: 3.0144... Val Loss: 2.9629
Epoch: 1/20... Step: 40... Loss: 3.0068... Val Loss: 2.9574
Epoch: 1/20... Step: 50... Loss: 2.9776... Val Loss: 2.9588
Epoch: 1/20... Step: 60... Loss: 3.0133... Val Loss: 2.9612
Epoch: 1/20... Step: 70... Loss: 2.9932... Val Loss: 2.9619
Epoch: 1/20... Step: 80... Loss: 2.9580... Val Loss: 2.9553
Epoch: 1/20... Step: 90... Loss: 2.9512... Val Loss: 2.9535
Epoch: 1/20... Step: 100... Loss: 2.9550... Val Loss: 2.9502
Epoch: 1/20... Step: 110... Loss: 2.9439... Val Loss: 2.9475
Epoch: 1/20... Step: 120... Loss: 2.9426... Val Loss: 2.9378
Epoch: 1/20... Step: 130... Loss: 2.9198... Val Loss: 2.9191
Epoch: 1/20... Step: 140... Loss: 2.8840... Val Loss: 2.8735
Epoch: 1/20... Step: 150... Loss: 2.8228... Val Loss: 2.7868
Epoch: 1/20... Step: 160... Loss: 2.7575... Val Loss: 2.6969
Epoch: 1/20... Step: 170... Loss:

## Getting the best model

Set your hyperparameters to get the best performance, and watch the training and validation losses.

## Checkpoint

After training, save the model to load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, I'll want to sample from it and make predictions about next characters! To sample, pass in a character and have the network predict the next character. Then take that character, pass it back in, and get another predicted character. Just keep doing this and generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

The predictions come from a categorical probability distribution over all the possible characters. The sample text can be made more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. 

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating song 

Typically I'll want to prime the network to build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [0]:
def SongGenerate(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

Song Generation by the Neural Network

In [29]:
print(SongGenerate(net, 1000, prime='lyrics', top_k=5))

lyrics over my best friend
share it so i don't want to see
i'm a liar a song and you'll never be the only one
i'm trying to late this sun
as i came at home baby
when we're stayin together
a minute of the sun
in the core of me i know this is a love that could say
we went from about the love i had
it's all over it's only love
well i wanna feel it's love with you
and it's soon that we're not forget
it's all alone i wanna be and out of you
wandaning you knew it was you
i wanna love you i'll never keep being alone
i've been down tonight
i'll give you love my first love
you'll be my strength to break all my mind
the story of your singing treated things
and the truth i knew i'd leave you
i've been tripping you're now it feels a lonely lover
you've got love but the time is real
it's gonna be all right
yet and there's a love and the world
i'm never gonna be stronger
you can't help it
you can take a lie
tell me what you're standing for
shart with the world with a chance to see
who is your smile?